# Excercise

We'll be solving below excercises learned from 1-bigram notebook,

1. train a trigram language model, i.e. take two characters as an input to predict the 3rd one. Feel free to use either counting or a neural net. Evaluate the loss; Did it improve over a bigram model?
2. split up the dataset randomly into 80% train set, 10% dev set, 10% test set. Train the bigram and trigram models only on the training set. Evaluate them on dev and test splits. What can you see?
3. use the dev set to tune the strength of smoothing (or regularization) for the trigram model - i.e. try many possibilities and see which one works best based on the dev set loss. What patterns can you see in the train and dev set loss as you tune this strength? Take the best setting of the smoothing and evaluate on the test set once and at the end. How good of a loss do you achieve?
4. we saw that our 1-hot vectors merely select a row of W, so producing these vectors explicitly feels wasteful. Can you delete our use of F.one_hot in favor of simply indexing into rows of W?
5. look up and use F.cross_entropy instead. You should achieve the same result. Can you think of why we'd prefer to use F.cross_entropy instead?

## Excercise 1: Trigram model

In [1]:
# Importing dataset
words = open('names.txt', 'r').read().splitlines()
words[:10]

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']

In [6]:
words[0][1:]

'mma'

In [34]:
chars = sorted(list(set(''.join(words))))
stoi = {s: i+1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}

In [39]:
import torch
import torch.nn.functional as F

In [128]:
xs1, xs2, ys = [], [], []
for word in words:
    chs = ['.'] + list(word) + ['.']
    for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        ix3 = stoi[ch3]
        xs1.append(ix1)
        xs2.append(ix2)
        ys.append(ix3)
xs1 = torch.tensor(xs1)
xs2 = torch.tensor(xs2)
ys = torch.tensor(ys)
num = xs1.nelement()
print(f"Number of examples: {num}")

Number of examples: 196113


For trigram, we now have two inputs and one outpt. So in the forward pass, we'll initialize two weights(neurons) and perform a sum of xs1 @ W1 + xs2 @ W2. Assuming weighted sum of inputs is logits.

In [129]:
# Initialize weights
g = torch.Generator().manual_seed(42)
W1 = torch.randn((27, 27), generator=g, requires_grad=True)
W2 = torch.randn((27, 27), generator=g, requires_grad=True)

In [130]:
W1.shape, W2.shape

(torch.Size([27, 27]), torch.Size([27, 27]))

In [131]:
for k in range(100):

    x1enc = F.one_hot(xs1, num_classes=27).float()
    x2enc = F.one_hot(xs2, num_classes=27).float()
    logits = x1enc @ W1 + x2enc @ W2
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdims=True)
    loss = -probs[torch.arange(num), ys].log().mean()
    print(loss.item())

    W1.grad = None
    W2.grad = None
    loss.backward()

    W1.data += -50 * W1.grad
    W2.data += -50 * W2.grad
    

4.03549337387085
3.38352632522583
3.066603422164917
2.884695053100586
2.772024393081665
2.6933724880218506
2.634974241256714
2.5894908905029297
2.552961587905884
2.5229411125183105
2.4978718757629395
2.4766249656677246
2.458418130874634
2.4426538944244385
2.4289000034332275
2.416811466217041
2.4061248302459717
2.3966212272644043
2.388127565383911
2.3804965019226074
2.373607635498047
2.3673579692840576
2.3616628646850586
2.356449842453003
2.3516578674316406
2.347235918045044
2.3431406021118164
2.339334011077881
2.3357856273651123
2.332468032836914
2.329357862472534
2.3264355659484863
2.323683500289917
2.3210864067077637
2.318631649017334
2.3163068294525146
2.3141021728515625
2.3120081424713135
2.310016632080078
2.3081207275390625
2.3063132762908936
2.3045878410339355
2.3029398918151855
2.301363945007324
2.2998554706573486
2.298410654067993
2.297025203704834
2.2956955432891846
2.2944185733795166
2.29319167137146
2.292011260986328
2.2908754348754883
2.289781332015991
2.288726806640625
2.2

The loss is reduced to 2.26 compared to 2.47 loss of trigram model.

In [172]:
for i in range(5):

    out = []
    ix1 = 0
    ix2 = 1
    while True:
        x1enc = F.one_hot(torch.tensor([ix1]), num_classes=27).float()
        x2enc = F.one_hot(torch.tensor([ix2]), num_classes=27).float()
        logits = x1enc @ W1 + x2enc @ W2
        counts = logits.exp()
        p = counts / counts.sum(1, keepdims=True)
        next_ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        out.append(itos[next_ix])
        ix1 = ix2
        ix2 = next_ix
        if ix2 == 0:
            break
    print(''.join(out))

rsipel.
lyania.
ha.
roon.
rlockasmily.


We're getting something like names but some of them are as bad as bigram model. But some improvement can be seen.